In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import pandas as pd
import pickle
from PopSynthesis.Methods.IPSF.const import (
    processed_dir,
    data_dir,
    PP_ATTS,
    POOL_SIZE,
    NOT_INCLUDED_IN_BN_LEARN,
)
from PopSynthesis.Methods.IPSF.CSP.operations.convert_seeds import convert_seeds_to_pairs, pair_states_dict
from PopSynthesis.Methods.IPSF.utils.pool_utils import create_pool
from PopSynthesis.DataProcessor.utils.seed.pp.process_relationships import MIN_GRANDPARENT_GRANDCHILD_GAP, MIN_PARENT_CHILD_GAP
from PopSynthesis.Methods.IPSF.CSP.operations.extra_filters import filter_mismatch_hhsz, filter_paired_pool_agegr

In [3]:
# get the data
hh_marg = pd.read_csv(data_dir / "hh_marginals_ipu.csv", header=[0, 1])
hh_seed = pd.read_csv(data_dir / "hh_sample_ipu.csv").drop(columns=["sample_geog"])
pp_seed = pd.read_csv(data_dir / "pp_sample_ipu.csv").drop(columns=["sample_geog"])
with open(processed_dir / "dict_hh_states.pickle", "rb") as handle:
    hh_att_state = pickle.load(handle)
with open(processed_dir / "dict_pp_states.pickle", "rb") as handle:
    pp_att_state = pickle.load(handle)

In [4]:
hh_marg = hh_marg.drop(columns=hh_marg.columns[hh_marg.columns.get_level_values(0)=="sample_geog"][0])
# vars
rela_col = "relationship"
id_col = "serialno"
main_rela = "Main"
hh_tag = "HH"

In [5]:
# process seed
seed_pairs = convert_seeds_to_pairs(hh_seed, pp_seed, id_col, rela_col, main_rela)

In [6]:
# create pools
pools_ref = {}
for pair_name, pair_seed in seed_pairs.items():
    name1, name2 = pair_name.split("-")
    ori_states_1 = hh_att_state if name1 == hh_tag else pp_att_state
    ori_states_2 = pp_att_state # because the second one always people
    processed_states_ref = pair_states_dict(ori_states_1, ori_states_2, name1, name2)
    # we only need matching columns that we wish to process for BN
    # this excludes relationship and ids
    assert set(processed_states_ref.keys()) <= set(pair_seed.columns)
    to_filter_col = list(processed_states_ref.keys())
    if name1 == "HH":
        # special case to add the rela cols
        to_filter_col += list(pp_seed[rela_col].unique())
    filtered_seed = pair_seed[to_filter_col]
    pools_ref[pair_name] = create_pool(filtered_seed, state_names=processed_states_ref, pool_sz=POOL_SIZE)

Learn BN


  0%|          | 0/1000000 [00:00<?, ?it/s]

Doing the sampling


  0%|          | 0/18 [00:00<?, ?it/s]

Learn BN


...

  0%|          | 0/10 [00:00<?, ?it/s]

Learn BN


  0%|          | 0/1000000 [00:00<?, ?it/s]

Doing the sampling


  0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
pp_atts = list(set(PP_ATTS) - set(NOT_INCLUDED_IN_BN_LEARN))
all_rela = [x.split("-")[-1] for x in pools_ref.keys()]

In [8]:
# Rename HH-Main and filter mismatch by hhszie
rename_main = {x: f"{x}_{main_rela}" for x in pp_atts}
pools_ref[f"{hh_tag}-{main_rela}"] = pools_ref[f"{hh_tag}-{main_rela}"].rename(columns=rename_main)
pools_ref[f"{hh_tag}-{main_rela}"] = filter_mismatch_hhsz(
    pools_ref[f"{hh_tag}-{main_rela}"], "hhsize", all_rela
)

In [24]:
# Filter to remove cases of wrong age-group for these 4 special
for rela in ["Grandchild", "Child", "Parent", "Grandparent"]:
    min_gap = MIN_GRANDPARENT_GRANDCHILD_GAP if rela in ["Grandchild", "Grandparent"] else MIN_PARENT_CHILD_GAP
    main_age_col = f"age_{main_rela}"
    rela_age_col = f"age_{rela}"
    older_age_col = main_age_col if rela in ["Grandchild", "Child"] else rela_age_col
    younger_age_col = main_age_col if rela in ["Parent", "Grandparent"] else rela_age_col
    assert younger_age_col != older_age_col
    pools_ref[f"{main_rela}-{rela}"] = filter_paired_pool_agegr(pool=pools_ref[f"{main_rela}-{rela}"], agegr_col_younger=younger_age_col, agegr_col_older=older_age_col, min_gap=min_gap)

In [35]:
with open(processed_dir / "dict_pool_pairs.pickle", "wb") as handle:
    pickle.dump(pools_ref, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
pools_ref["Main-Parent"]

,sex_Main,sex_Parent,anywork_Main,anywork_Parent,persinc_Main,persinc_Parent,nolicence_Main,nolicence_Parent,age_Main,age_Parent
0,M,M,Y,N,$1500-1999 p.w.,$400-599 p.w.,Some Licence,Some Licence,20-29,80-89
1,M,F,Y,N,$800-999 p.w.,$400-599 p.w.,Some Licence,Some Licence,20-29,60-69
2,M,M,Y,Y,$1000-1249 p.w.,$300-399 p.w.,Some Licence,Some Licence,20-29,50-59
3,F,F,Y,N,$1250-1499 p.w.,$600-799 p.w.,No Licence,Some Licence,50-59,80-89
4,M,F,Y,N,$1500-1999 p.w.,$300-399 p.w.,Some Licence,No Licence,20-29,70-79
...,...,...,...,...,...,...,...,...,...,...
4999995,F,M,Y,N,$1000-1249 p.w.,Zero Income,Some Licence,Some Licence,20-29,60-69
4999996,M,F,Y,N,$600-799 p.w.,$300-399 p.w.,Some Licence,Some Licence,40-49,60-69
4999997,M,F,Y,N,$600-799 p.w.,$200-299 p.w.,Some Licence,Some Licence,30-39,60-69
4999998,M,F,Y,Y,$800-999 p.w.,$400-599 p.w.,Some Licence,Some Licence,20-29,50-59
